In [15]:
import numpy as np
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import chi2, SelectKBest, SequentialFeatureSelector, SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [16]:
ds = pd.read_csv('data/SMS.tsv',sep='\t')
ds.head(5)

,class,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [17]:
ds['class'] = pd.factorize(ds['class'])[0] # ham - 0, spam - 1

In [18]:
y = ds['class']

In [19]:
X_text = []
for s in ds['text']:
    s = s.translate(str.maketrans('', '', string.punctuation))
    s = re.sub('\s+', ' ', s)
    s = re.sub('\d+', '', s)
    X_text.append(s)

In [28]:
vectorizer = TfidfVectorizer(stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(X_text)
X = pd.DataFrame(X.toarray(), columns = vectorizer.get_feature_names_out())
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

Встроенный

In [29]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

RandomForestClassifier()

In [30]:
coef=pd.Series(rf.feature_importances_, X.columns).sort_values(ascending=False)
print(coef)

call          0.037332
free          0.026076
txt           0.023958
claim         0.023080
mobile        0.019034
                ...   
joinedhope    0.000000
joined        0.000000
jogging       0.000000
jog           0.000000
〨ud           0.000000
Length: 8479, dtype: float64


In [31]:
coef_emb = pd.Series(rf.feature_importances_, X.columns).sort_values()

обёртка

In [ ]:
X_train_filt = X_train

while len(X_train_filt.columns) > 100:
    tree = DecisionTreeClassifier()
    tree.fit(X_train_filt, y_train)
    coef=pd.Series(tree.feature_importances_, X_train_filt.columns).sort_values()
    for i in range(10):
        X_train_filt = X_train_filt.drop(coef.index[i], axis=1)  

coef_wrap = coef

In [ ]:
coef_wrap

фильтр

In [ ]:
X_train_filter = X_train

corr = X_train_filter.corr(method='pearson')

corr_class = corr['class']
coef_filt = corr_class.sort_values()

In [ ]:
coef_filt